In [1]:
import pandas as pd 
import numpy as np
import pandasql as ps

In [2]:
d1 = { 'name': ['A','A','A','B','B','B','C','C','C'],
       'subject': ['Maths','science','social','Maths','science','social','Maths','science','social'],
		'marks': [91,20,91,35,38,42,91,38,90]}
df = pd.DataFrame(d1)
df

,name,subject,marks
0,A,Maths,91
1,A,science,20
2,A,social,91
3,B,Maths,35
4,B,science,38
5,B,social,42
6,C,Maths,91
7,C,science,38
8,C,social,90


##  Just a basic SELECT clause
`
SELECT name,marks FROM df;
`

In [3]:
q1 = '''SELECT name,marks FROM df;'''
print(ps.sqldf(q1,locals()))

  name  marks
0    A     91
1    A     20
2    A     91
3    B     35
4    B     38
5    B     42
6    C     91
7    C     38
8    C     90


In [4]:
print(df[['name','marks']])

  name  marks
0    A     91
1    A     20
2    A     91
3    B     35
4    B     38
5    B     42
6    C     91
7    C     38
8    C     90


##  SELECT with WHERE
`
SELECT name,marks FROM df WHERE marks < 35;
`

In [5]:
q1 = '''SELECT name,marks FROM df WHERE marks < 35'''
print(ps.sqldf(q1,locals()))

  name  marks
0    A     20


In [6]:
df.loc[df['marks'] < 35,['name','marks']]

,name,marks
1,A,20


##  SELECT with WHERE, AND, OR (multiple conditions)
`SELECT name,marks FROM df WHERE marks < 35 OR name ='B';
`

In [7]:
q1 = '''SELECT name,marks FROM df WHERE marks < 35 OR name ='B'; '''
print(ps.sqldf(q1,locals()))

  name  marks
0    A     20
1    B     35
2    B     38
3    B     42


In [8]:
df.loc[(df['marks'] < 35) | (df['name'] == 'B'), ['name', 'marks']]


,name,marks
1,A,20
3,B,35
4,B,38
5,B,42


## SELECT - WHERE - IN clauses
`
SELECT name,marks FROM df WHERE marks in (35,40,42,91);
`

In [9]:
q1 = ''' SELECT name,marks 
         FROM df 
         WHERE marks IN (35,40,42,91)'''
print(ps.sqldf(q1,locals()))

  name  marks
0    A     91
1    A     91
2    B     35
3    B     42
4    C     91


In [10]:
df.loc[df['marks'].isin([35,40,42,91]),['name','marks']]

,name,marks
0,A,91
2,A,91
3,B,35
5,B,42
6,C,91


##  Arithmetic operation with in SELECT  Clause
`SELECT name,marks, marks*2 as newMarks
FROM df ;
`

In [11]:
q1 = ''' SELECT name,marks, marks*2 as newMarks
         FROM df '''
print(ps.sqldf(q1,locals()))



  name  marks  newMarks
0    A     91       182
1    A     20        40
2    A     91       182
3    B     35        70
4    B     38        76
5    B     42        84
6    C     91       182
7    C     38        76
8    C     90       180


In [12]:
df['newMarks']= df['marks'] * 2

print(df[['name','marks','newMarks']])
#dropping newMarks column
df.drop('newMarks',axis=1,inplace=True)

  name  marks  newMarks
0    A     91       182
1    A     20        40
2    A     91       182
3    B     35        70
4    B     38        76
5    B     42        84
6    C     91       182
7    C     38        76
8    C     90       180


## SELECT - WHERE with NULL 
`
SELECT name,marks FROM df2 WHERE marks IS NULL;
`

In [13]:
d2 = { 'name': ['A','A','A','B','B','B','C','C','C'],
       'subject': ['Maths','science','social','Maths','science','social','Maths','science','social'],
		'marks': [91,20,np.nan,35,38,42,np.nan,38,90]}
df2 = pd.DataFrame(d2)
df2

,name,subject,marks
0,A,Maths,91.0
1,A,science,20.0
2,A,social,NaN
3,B,Maths,35.0
4,B,science,38.0
5,B,social,42.0
6,C,Maths,NaN
7,C,science,38.0
8,C,social,90.0


In [14]:
q1 = ''' SELECT * 
         FROM df2 
         WHERE marks IS NULL;'''
ps.sqldf(q1,locals())

,name,subject,marks
0,A,social,None
1,C,Maths,None


In [15]:
df2[df2['marks'].isna()]

,name,subject,marks
2,A,social,NaN
6,C,Maths,NaN


## JOINS

In [16]:
# Creating Dictionary
d = {'id': [1, 2, 3, 4],
	'val1': ['a', 'b', 'c', 'd']}

a = pd.DataFrame(d)

# Creating dictionary
d = {'id': [1, 2, 5, 6],
	'val1': ['p', 'q', 'r', 's']}
b = pd.DataFrame(d)

print(a)
print()
print(b)

   id val1
0   1    a
1   2    b
2   3    c
3   4    d

   id val1
0   1    p
1   2    q
2   5    r
3   6    s


### Inner Join
`SELECT a.*,b.* 
 FROM a
 INNER JOIN b 
  on a.id=b.id;
`

In [17]:
q1 = ''' SELECT a.*,b.val1 
         FROM a
         INNER JOIN b 
          on a.id=b.id;
          '''
ps.sqldf(q1,locals())

,id,val1,val1
0,1,a,p
1,2,b,q


In [18]:
# inner join
pd.merge(a, b, on='id', how='inner')


,id,val1_x,val1_y
0,1,a,p
1,2,b,q


### LEFT Join
`SELECT a.*,b.* 
 FROM a
 LEFT JOIN b 
  on a.id=b.id;
`

In [19]:
q1 = ''' SELECT a.*,b.*
         FROM a
         LEFT JOIN b 
          on a.id=b.id;
          '''
ps.sqldf(q1,locals())

,id,val1,id,val1
0,1,a,1.0,p
1,2,b,2.0,q
2,3,c,NaN,None
3,4,d,NaN,None


In [20]:
# left outer join
pd.merge(a, b, on='id', how='left')

,id,val1_x,val1_y
0,1,a,p
1,2,b,q
2,3,c,NaN
3,4,d,NaN


### RIGHT Join
`SELECT a.*,b.* 
 FROM a
 RIGHT JOIN b 
  on a.id=b.id;
`

In [21]:
q1 = ''' SELECT a.*,b.*
         FROM a
         RIGHT JOIN b 
          on a.id=b.id;
          '''
# PandaSQLException: (sqlite3.OperationalError) RIGHT and FULL OUTER JOINs are not currently supported

In [22]:
# right outer join
pd.merge(a, b, on='id', how='right')

,id,val1_x,val1_y
0,1,a,p
1,2,b,q
2,5,NaN,r
3,6,NaN,s


### FULL OUTER Join
`SELECT a.*,b.* 
 FROM a
 FULL JOIN b 
  on a.id=b.id;
`

In [23]:
q1 = ''' SELECT a.*,b.*
         FROM a
         FULL JOIN b 
          on a.id=b.id;
          '''
# PandaSQLException: (sqlite3.OperationalError) RIGHT and FULL OUTER JOINs are not currently supported

In [24]:
# full outer join
pd.merge(a, b, on='id', how='outer')

,id,val1_x,val1_y
0,1,a,p
1,2,b,q
2,3,c,NaN
3,4,d,NaN
4,5,NaN,r
5,6,NaN,s


### CROSS Join - joining 2 or More table without any matching column is called as cross join
`SELECT a.*,b.* 
 FROM a, b;
`

In [25]:
q1 = ''' SELECT a.*,b.*
         FROM a, b ;
          '''
print(ps.sqldf(q1,locals()))

    id val1  id val1
0    1    a   1    p
1    1    a   2    q
2    1    a   5    r
3    1    a   6    s
4    2    b   1    p
5    2    b   2    q
6    2    b   5    r
7    2    b   6    s
8    3    c   1    p
9    3    c   2    q
10   3    c   5    r
11   3    c   6    s
12   4    d   1    p
13   4    d   2    q
14   4    d   5    r
15   4    d   6    s


In [26]:
# cross join
print(pd.merge(a, b, how='cross'))

    id_x val1_x  id_y val1_y
0      1      a     1      p
1      1      a     2      q
2      1      a     5      r
3      1      a     6      s
4      2      b     1      p
5      2      b     2      q
6      2      b     5      r
7      2      b     6      s
8      3      c     1      p
9      3      c     2      q
10     3      c     5      r
11     3      c     6      s
12     4      d     1      p
13     4      d     2      q
14     4      d     5      r
15     4      d     6      s


## Horizontal Concatenation - Not present in SQL

In [27]:
# index join
pd.merge(a, b, left_index=True, right_index=True)

,id_x,val1_x,id_y,val1_y
0,1,a,1,p
1,2,b,2,q
2,3,c,5,r
3,4,d,6,s


In [28]:
pd.concat([a,b],axis='columns')

,id,val1,id,val1
0,1,a,1,p
1,2,b,2,q
2,3,c,5,r
3,4,d,6,s


## UNION All - Duplicates will be included from result data
`SELECT * 
 FROM c
 UNION ALL
 SELECT * 
 FROM d
 ;
`

In [29]:
# Creating Dictionary
d = {'id': [1, 2,3],
	'val1': ['a', 'b', 'c']}

c = pd.DataFrame(d)

# Creating dictionary
d = {'id': [1, 2,3],
	'val1': ['a', 'b', 'd']}
d= pd.DataFrame(d)

In [30]:
q1 = '''
 SELECT * 
 FROM c
 UNION ALL
 SELECT * 
 FROM d
'''
print(ps.sqldf(q1,locals()))

   id val1
0   1    a
1   2    b
2   3    c
3   1    a
4   2    b
5   3    d


In [31]:
pd.concat([c,d],axis='index')

,id,val1
0,1,a
1,2,b
2,3,c
0,1,a
1,2,b
2,3,d


## UNION - Duplicates will be excluded from result data
`SELECT * 
 FROM c
 UNION
 SELECT * 
 FROM d
 ;
`

In [32]:
q1 = '''
 SELECT * 
 FROM c
 UNION 
 SELECT * 
 FROM d
'''
print(ps.sqldf(q1,locals()))

   id val1
0   1    a
1   2    b
2   3    c
3   3    d


In [33]:
pd.concat([c,d],axis='index').drop_duplicates()

,id,val1
0,1,a
1,2,b
2,3,c
2,3,d


## GROUP BY 

In [34]:
df

,name,subject,marks
0,A,Maths,91
1,A,science,20
2,A,social,91
3,B,Maths,35
4,B,science,38
5,B,social,42
6,C,Maths,91
7,C,science,38
8,C,social,90


`
SELECT name,count(*) as no_of_subjects 
 FROM df
 GROUP BY name
 `

In [35]:
q1 = '''
 SELECT name,count(*) as no_of_subjects 
 FROM df
 GROUP BY name
'''
print(ps.sqldf(q1,locals()))

  name  no_of_subjects
0    A               3
1    B               3
2    C               3


In [36]:
df.groupby('name').size()

name
A    3
B    3
C    3
dtype: int64

`
SELECT name,count(subject) as no_of_subjects 
 FROM df
 GROUP BY name
 `

In [37]:
q1 = '''
 SELECT name,count(subject) as no_of_subjects 
 FROM df
 GROUP BY name
'''
print(ps.sqldf(q1,locals()))

  name  no_of_subjects
0    A               3
1    B               3
2    C               3


In [38]:
df.groupby('name')['subject'].count()

name
A    3
B    3
C    3
Name: subject, dtype: int64

`
SELECT name,
       count(subject) as no_of_subjects,
       sum(marks) as total_marks
 FROM df
 GROUP BY name
 `

In [39]:
q1 = '''
 SELECT name,
       count(subject) as no_of_subjects,
       sum(marks) as total_marks
 FROM df
 GROUP BY name
'''
print(ps.sqldf(q1,locals()))

  name  no_of_subjects  total_marks
0    A               3          202
1    B               3          115
2    C               3          219


In [40]:
df.groupby('name').agg({'subject': 'count', 'marks': 'sum'}).rename(columns={'subject': 'no_of_subjects'})


,no_of_subjects,marks
name,,
A,3,202
B,3,115
C,3,219


`
SELECT name,
       count(subject) as no_of_subjects,
       sum(marks) as total_marks,
       max(marks) as highest_marks
 FROM df
 GROUP BY name
 `

In [41]:
q1 = '''
 SELECT name,
       count(subject) as no_of_subjects,
       sum(marks) as total_marks,
       max(marks) as highest_marks
 FROM df
 GROUP BY name
'''
print(ps.sqldf(q1,locals()))

  name  no_of_subjects  total_marks  highest_marks
0    A               3          202             91
1    B               3          115             42
2    C               3          219             91


In [42]:
# group by name and apply count, sum and max functions on subject and marks columns
df.groupby('name').agg(no_of_subjects=('subject', 'count'), total_marks=('marks', 'sum'), highest_marks=('marks', 'max')).reset_index()


,name,no_of_subjects,total_marks,highest_marks
0,A,3,202,91
1,B,3,115,42
2,C,3,219,91


## Window Functions - Row_Number(), Rank(), Dense_Rank()

In [43]:
q1 ='''select name,subject,marks,
row_number() over(partition by name order by marks desc ) as row_n,
rank() over(partition by name order by marks desc) as rnk,
dense_rank() over(partition by name order by marks desc) as dense_rnk
from df'''
print(ps.sqldf(q1,locals()))


  name  subject  marks  row_n  rnk  dense_rnk
0    A    Maths     91      1    1          1
1    A   social     91      2    1          1
2    A  science     20      3    3          2
3    B   social     42      1    1          1
4    B  science     38      2    2          2
5    B    Maths     35      3    3          3
6    C    Maths     91      1    1          1
7    C   social     90      2    2          2
8    C  science     38      3    3          3


In [44]:
# create a new column for row number
df['row_n1'] = df.groupby('name')['marks'].rank(method='first', ascending=False)

# create a new column for rank
df['rnk1'] = df.groupby('name')['marks'].rank(method='min', ascending=False)

# create a new column for dense rank
df['dense_rnk1'] = df.groupby('name')['marks'].rank(method='dense', ascending=False)

# values float to integer
df[['row_n1', 'rnk1', 'dense_rnk1']] = df[['row_n1', 'rnk1', 'dense_rnk1']].astype(int)

# print the dataframe
print(df)


  name  subject  marks  row_n1  rnk1  dense_rnk1
0    A    Maths     91       1     1           1
1    A  science     20       3     3           2
2    A   social     91       2     1           1
3    B    Maths     35       3     3           3
4    B  science     38       2     2           2
5    B   social     42       1     1           1
6    C    Maths     91       1     1           1
7    C  science     38       3     3           3
8    C   social     90       2     2           2


 ##### Author: &lt; Shafi Mohiddin Shaik&gt; <hr> LinkedIn : [Click to get redirected to Shafi's Linkedin Page]( https://www.linkedin.com/in/shafi-s-0279651b0/)